In [2]:
import pandas as pd
import re

# Exemplo de dados carregados diretamente (você precisará substituir pelo caminho correto do arquivo)
with open('data/train.ft.txt', 'r', encoding='utf-8') as f:
    raw_data = f.readlines()

# Função para separar o rótulo e o texto da revisão
def process_review_line(line):
    # Separar rótulo do texto
    match = re.match(r'(__label__\d) (.+)', line)
    if match:
        label = match.group(1)
        review = match.group(2)
        return label, review
    return None, None

# Processar todas as linhas e organizar em listas
labels = []
reviews = []

for line in raw_data:
    label, review = process_review_line(line.strip())
    if label and review:
        labels.append(label)
        reviews.append(review)

# Criar DataFrame organizado
df_train = pd.DataFrame({'label': labels, 'review': reviews})

# Exibir as primeiras linhas do DataFrame
print(df_train.head())

        label                                             review
0  __label__2  Stuning even for the non-gamer: This sound tra...
1  __label__2  The best soundtrack ever to anything.: I'm rea...
2  __label__2  Amazing!: This soundtrack is my favorite music...
3  __label__2  Excellent Soundtrack: I truly like this soundt...
4  __label__2  Remember, Pull Your Jaw Off The Floor After He...


In [3]:
# Mapeando os rótulos para valores numéricos
df_train['label'] = df_train['label'].map({'__label__1': 1, '__label__2': 2})
df_train.head()

,label,review
0,2,Stuning even for the non-gamer: This sound tra...
1,2,The best soundtrack ever to anything.: I'm rea...
2,2,Amazing!: This soundtrack is my favorite music...
3,2,Excellent Soundtrack: I truly like this soundt...
4,2,"Remember, Pull Your Jaw Off The Floor After He..."


In [4]:
import pandas as pd
import re

# Exemplo de dados carregados diretamente (você precisará substituir pelo caminho correto do arquivo)
with open('data/test.ft.txt', 'r', encoding='utf-8') as f:
    raw_data = f.readlines()

# Função para separar o rótulo e o texto da revisão
def process_review_line(line):
    # Separar rótulo do texto
    match = re.match(r'(__label__\d) (.+)', line)
    if match:
        label = match.group(1)
        review = match.group(2)
        return label, review
    return None, None

# Processar todas as linhas e organizar em listas
labels = []
reviews = []

for line in raw_data:
    label, review = process_review_line(line.strip())
    if label and review:
        labels.append(label)
        reviews.append(review)

# Criar DataFrame organizado
df_test = pd.DataFrame({'label': labels, 'review': reviews})

# Exibir as primeiras linhas do DataFrame
print(df_test.head())

        label                                             review
0  __label__2  Great CD: My lovely Pat has one of the GREAT v...
1  __label__2  One of the best game music soundtracks - for a...
2  __label__1  Batteries died within a year ...: I bought thi...
3  __label__2  works fine, but Maha Energy is better: Check o...
4  __label__2  Great for the non-audiophile: Reviewed quite a...


In [5]:
# Mapeando os rótulos para valores numéricos
df_test['label'] = df_test['label'].map({'__label__1': 1, '__label__2': 2})
df_test.head()

,label,review
0,2,Great CD: My lovely Pat has one of the GREAT v...
1,2,One of the best game music soundtracks - for a...
2,1,Batteries died within a year ...: I bought thi...
3,2,"works fine, but Maha Energy is better: Check o..."
4,2,Great for the non-audiophile: Reviewed quite a...


In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Baixar pacotes do nltk necessários
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Inicializar lematizador e obter lista de stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Função de tokenização e lematização
def tokenize(text):
    expression = r"\w+"
    match = re.findall(expression, text)
    tokens = [lemmatizer.lemmatize(m.lower()) for m in match if m.lower() not in stop_words]
    return tokens

# Aplicar ao DataFrame
df_train['tokens'] = df_train['review'].apply(tokenize)
print(df_train['tokens'].head())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joaom\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\joaom\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\joaom\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


0    [stuning, even, non, gamer, sound, track, beau...
1    [best, soundtrack, ever, anything, reading, lo...
2    [amazing, soundtrack, favorite, music, time, h...
3    [excellent, soundtrack, truly, like, soundtrac...
4    [remember, pull, jaw, floor, hearing, played, ...
Name: tokens, dtype: object


In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(df_train['review'], df_train['label'], test_size=0.2, random_state=42)

# Definir pipeline com Bag-of-Words e Naive Bayes
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('clf', MultinomialNB())
])

# Treinar o modelo
pipeline.fit(X_train, y_train)

# Avaliar o modelo
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           1       0.84      0.85      0.85    359759
           2       0.85      0.84      0.84    360241

    accuracy                           0.85    720000
   macro avg       0.85      0.85      0.85    720000
weighted avg       0.85      0.85      0.85    720000



In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
#arrumar o problema de nao ter importado nb 


# Exemplo de uso do TfidfVectorizer
vectorizer = TfidfVectorizer()

# Transformar os textos em vetores numéricos
X_train_tfidf = vectorizer.fit_transform(X_train)  # X_train é o conjunto de reviews de treino
X_test_tfidf = vectorizer.transform(X_test)  # Transformar os textos de teste também

# Agora você pode treinar o modelo usando os dados vetorizados
nb.fit(X_train_tfidf, y_train)

# Fazer previsões
y_pred_nb = nb.predict(X_test_tfidf)

# Avaliar o modelo
from sklearn.metrics import balanced_accuracy_score
score_nb = balanced_accuracy_score(y_test, y_pred_nb)
print(f"Balanced Accuracy Naive Bayes: {score_nb}")


KeyboardInterrupt: 